<a href="https://colab.research.google.com/github/kaitlynlaii/NUS_DATATHON_SINGLIFE_NUS_124/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The cell below is for you to keep track of the libraries used and install those libraries quickly

Ensure that the proper library names are used and the syntax of %pip install PACKAGE_NAME is followed



In [ ]:
#%pip install pandas
#%pip install matplotlib
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME

DO NOT CHANGE the filepath variable

Instead, create a folder named 'data' in your current working directory and

have the .parquet file inside that. A relative path must be used when loading data into pandas

In [ ]:
# Can have as many cells as you want for code
import pandas as pd
import numpy as np
filepath = "./data/catB_train.parquet"
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

ALL Code for machine learning and dataset analysis should be entered below.

Ensure that your code is clear and readable.

Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.
In [ ]:


In [ ]:
df = pd.read_parquet(filepath)

The cell below is NOT to be removed

The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list).

It is recommended to test the function out prior to submission

The hidden_data parsed into the function below will have the same layout columns wise as the dataset SENT to you

Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below
In [ ]:


In [ ]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform.

All relevant code MUST be included in this function.'''
    df = hidden_data
    # Convert target col to 0 or 1
    df["f_purchase_lh"] = df["f_purchase_lh"].fillna(0)
    # Set the threshold for missing values (50%)
    threshold = 0.5 * len(df)

    # Drop columns with 50% or more missing values
    df = df.dropna(axis=1, thresh=threshold)

    # Drop clntnum, ctrycode and date columns
    df = df.drop(columns=['clntnum', 'min_occ_date', 'cltdob_fix', 'ctrycode_desc'])
    df_columns_obj = df.select_dtypes(include='object').columns.tolist()

    # fill missing obj values with mode
    for title in df_columns_obj:
      df[title] = df[title].fillna(df[title].mode()[0])

    ### Manually change the object datatypes to integers / floats types
    # all ape/sumins/prempaid > float
    to_float = [col for col in df.columns if 'ape' in col or 'sumins' in col or 'prempaid' in col]
    df[to_float] = df[to_float].astype(float)

    # all n_months, hh_20, pop_20 > int
    to_int = [col for col in df.columns if 'n_months' in col]
    to_int.extend(['hh_20','pop_20'])
    df[to_int] = df[to_int].astype('int64')

    # race, clnt type, stat, sex > nominal, one hot encoding
    to_one_hot = ['race_desc', 'clttype', 'stat_flag', 'cltsex_fix']
    df = pd.get_dummies(df, columns=to_one_hot)

    # hh_size, annual income > ordinal encoding
    from sklearn.preprocessing import OrdinalEncoder
    to_ordinal = ['hh_size_est', 'annual_income_est']
    categories = [['0','1', '2', '3', '4', '>4'], ['E.BELOW30K','D.30K-60K','C.60K-100K' ,'B.100K-200K','A.ABOVE200K']]
    encoder = OrdinalEncoder(categories=categories)
    df[to_ordinal] = encoder.fit_transform(df[to_ordinal])

    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
    df[numeric_cols] = df[numeric_cols].apply(lambda x: x.fillna(x.median()))

    y = df["f_purchase_lh"]
    X = df.drop(columns=['f_purchase_lh'])

    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])

    from sklearn.model_selection import train_test_split
    from collections import Counter

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

    selected = ["n_months_last_bought_gi",
                  "is_valid_email",
                  "n_months_last_bought_products",
                  "f_ever_bought_gi",
                  "is_valid_dm",
                  "f_mindef_mha",
                  "f_retail",
                  "hh_size",
                  "pop_20" ]
    X_train = X_train[selected]
    X_val = X_val[selected]

    from collections import Counter
    from imblearn.over_sampling import SMOTE
    # SMOTE on training set only
    X_train, y_train = SMOTE().fit_resample(X_train, y_train)

    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    from sklearn.ensemble import RandomForestClassifier
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)

    y_pred_rf = rf_model.predict(X_val)

    # Evaluate the Random Forest model
    accuracy_rf = accuracy_score(y_val, y_pred_rf)
    conf_matrix_rf = confusion_matrix(y_val, y_pred_rf)
    classification_rep_rf = classification_report(y_val, y_pred_rf)

    result = [classification_rep_rf]
    return result